In [ ]:
%matplotlib inline
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
from torch.autograd import Variable
import nibabel as nib
import pandas as pd
from torch.utils import data
import random
from sklearn.model_selection import KFold
import argparse
import os
import time
import sys
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit
from scipy.spatial import distance_matrix
import pathlib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,balanced_accuracy_score,precision_score,f1_score,silhouette_score,recall_score,classification_report,confusion_matrix,roc_curve
from scipy.stats import ttest_ind
from mpl_toolkits.mplot3d import Axes3D   
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.decomposition import PCA
import seaborn as sns
from IPython.display import display, HTML
from scipy.stats import spearmanr,pearsonr, ttest_ind
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib,statsmodels
import datetime
from sklearn.linear_model import LinearRegression
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '#notebook-container {padding: 0px;}'
    '</style>'
))

In [ ]:
pt = 0.05

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)


def ln(name, X, y):
    X2 = sm.add_constant(X)
    est = sm.RLM(y, X2, M=sm.robust.norms.HuberT())  
#     est = sm.OLS(y, X2)
    est2 = est.fit()
    
    s1 = est2.summary2(float_format='%.2f')
#     print(s1.tables[1])
#     if s1.tables[1].iloc[1,3] > pt:
#         print(name + " p > " + str(pt) + ", not significant")
#     else:
#     print(name + " p, coef = {:.1e}".format(s1.tables[1].iloc[1,3]),',', np.round(s1.tables[1].iloc[1,0],4))
    s2 = est2.summary2().tables[1]
    return s2.iloc[0][0], s2.iloc[1][0],s1.tables[1].iloc[1,3]

def get_r2(X,y):
    model = LinearRegression()
    model.fit(X, y)
    return round(model.score(X, y),2)


def get_test(df, score):
    dfi = df.copy()
    if score == 'MMSE':  # lower = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 30

    if score =='ADAS13': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 100
        
    if score =='ADAS11': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 70
    

        
    if score =='ADASQ4': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 10
        
    if score =='MOCA': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 30
        
    if score =='TAU': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b = b.replace('<80', 0)
        b = b.replace('>1300', 1300)
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 1000
        
    if score =='PTAU': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b = b.replace('<8', 8)
        b = b.replace('>120', 120)
        b[score] = b[score].astype(float)
        min_ = 8
        max_ = 103
        
    if score =='ABETA': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b = b.replace('<200', 200)
        b = b.replace('>1700', 1701)
        b[score] = b[score].astype(float)
        min_ = 200
        max_ = 1700
        
    if score =='APOE4': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 2
    
    if score =='qPTAU': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 1
        
    if score == 'Hippocampus': 
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score]*(1/1000)
        min_ = 0
        max_ = 20
        
    if score =='TRABSCOR': 
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = 0
        max_ = 299
    
    if score =='WMH': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_3.5D_2020_DLWML_Features_DerivedVolumes.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score]/1000
        b[score] = b[score]**(1/3)
        min_ = 0
        max_ = 50
        
    if score =='mPACCdigit': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        min_ = -60
        max_ = 10
        
    if score =='CVANUMBER': # higher = severe
        b = pd.read_csv("/home/dw/Desktop/DemCLF/model/results/task2_T1_ADVDLBD_Mlabel_10fold_NoHarm_3DSminmax_OTF/resnet18_64_linear_mlr0.0001_ep300_batch8_ASL_gn2_gp1_c0.2_sigma0.1/prob_ADNI_TW_all2.csv")[["mri_ID", 'CVANUMBER']]
        min_ = 0
        max_ = 4
        
    if score =='META_TEMPORAL_SUVR': # subregion Weighted av1451 mean of meta-temporal region
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", 'META_TEMPORAL_SUVR']]
        min_ = 0
        max_ = 5
        
    if score =='syn': # subregion Weighted av1451 mean of meta-temporal region
        b = pd.read_csv("/home/dw/Desktop/ADNI/syn.csv")[["mri_ID", 'syn']]
        min_ = 0
        max_ = 1
    
    if score =='SPARE_TAU': # subregion Weighted av1451 mean of meta-temporal region
        b = pd.read_csv("/home/dw/Desktop/ADNI/available_tau_ADNI_SPARE_TAU.csv")[["mri_ID", 'SPARE_TAU']]
        min_ = -10
        max_ = 10
        
    if score =='ADNI_MEM': # subregion Weighted av1451 mean of meta-temporal region
        b = pd.read_csv("/home/dw/Desktop/ADNI/UWNPSYCHSUM_07Mar2024.csv")[["mri_ID", score]]
        min_ = -10
        max_ = 10
        
    if score =='ADNI_EF': # subregion Weighted av1451 mean of meta-temporal region
        b = pd.read_csv("/home/dw/Desktop/ADNI/UWNPSYCHSUM_07Mar2024.csv")[["mri_ID", score]]
        min_ = -10
        max_ = 10
        
    if score =='ADNI_LAN': # subregion Weighted av1451 mean of meta-temporal region
        b = pd.read_csv("/home/dw/Desktop/ADNI/UWNPSYCHSUM_07Mar2024.csv")[["mri_ID", score]]
        min_ = -10
        max_ = 10
        
    if score =='ADNI_VS': # subregion Weighted av1451 mean of meta-temporal region
        b = pd.read_csv("/home/dw/Desktop/ADNI/UWNPSYCHSUM_07Mar2024.csv")[["mri_ID", score]]
        min_ = -10
        max_ = 10
        
    if score =='CSF_alpha_syn':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 6
        
    if score =='PLASMA_NFL':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 270
    if score =='PLASMAPTAU181':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 100
    if score =='PLASMA_oAB':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 4000
    if score =='ratio_4240':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 1
    if score =='plasma_42':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 1
    if score =='CSF_ptau_ab42':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 0.99
    if score =='plasma_ptau_ab42':  
        b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", score]]
        b[score] = b[score].astype(float)
        min_ = 0
        max_ = 1
    

    dfi= pd.merge(dfi, b, on = 'mri_ID', how = 'left')
    dfi= dfi[(dfi[score]>=min_) & (dfi[score]<=max_)]
    return dfi

mm = 13

xlabel_font_size = 16
ylabel_font_size = 16


def ln_plot(df,dfp, group,scores): # ['MMSE','cubicWMHVOL','HIPPOVOL','TRAILB','BOSTON']:
    for score in scores:
        dff = df.copy()
        df_test = get_test(dff,score)
        print(score, len(df_test))

        df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD']) ) 
        df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD']))
        df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD']))

        pAD,pVD,pLBD, ymin, ymax = 'logit_probAD', 'logit_probVD','logit_probLBD', -5,5
        interAD,coefAD, pvAD,r2AD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interAD','coefAD', 'pvAD_fdr','r2AD']].values[0]
        interVD,coefVD, pvVD,r2VD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interVD','coefVD', 'pvVD_fdr','r2VD']].values[0]  
        interLBD,coefLBD,pvLBD,r2LBD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interLBD','coefLBD', 'pvLBD_fdr','r2LBD']].values[0] 

        print("AD:",pvAD, round(coefAD,2))
        print("VD:",pvVD, round(coefVD,2))
        print("LBD:",pvLBD, round(coefLBD,2))
        
        fig, axs = plt.subplots(ncols = 3, nrows = 1, figsize  = (11,3))
        axs = axs.flatten()
         
        yy1 = pAD
        yy2 = pVD
        yy3 = pLBD
        
        if score == "Hippocampus":
            xx = "Hippocampal Volume"
        elif score == "TRABSCOR":
            xx = "Trail B"
        elif score == "ADAS13":
            xx = "ADAS-Cog 13" 
        elif score == "ABETA":
            xx = "CSF A\u03B2" 
            
        if score == "WMH":    
            xx = r'$\sqrt[3]{WMH Volume}$'
        elif score == "META_TEMPORAL_SUVR":
            xx = "Meta Temporal SUVR"
        elif score == "SPARE_TAU":
            xx = "SPARE-TAU" 
        elif score == "PTAU":
            xx = "CSF pTau"
        elif score == "ADNI_MEM":
            xx = "ADNI-MEM"
        elif score == "ADNI_EF":
            xx = "ADNI-EF"
            
        elif score == "PLASMA_NFL":
            xx = "Plasma NFL"
        elif score == "PLASMAPTAU181":
            xx = "Plasma pTau"
            
        elif score == 'plasma_ptau_ab42':
            xx = "Plasma pTau/A\u03B2-42"
        elif score == 'CSF_ptau_ab42':
            xx = "CSF pTau/A\u03B2-42"
        else:
            xx = score
        
        for i, tp, yy, pv, coef, r2,inter in zip([0,1,2],['AD','VD','LBD'],[yy1,yy2,yy3],[pvAD,pvVD,pvLBD],
                                                 [coefAD,coefVD,coefLBD],[r2AD,r2VD,r2LBD],[interAD,interVD,interLBD]):
            dy = df_test[yy]
            dx = df_test[score]
            axs[i].scatter(dx, dy, s = 10, color = 'skyblue', alpha= 0.8)
            max_x = dx.max()
            min_x = dx.min()
            x = np.linspace(min_x, max_x,50)
            y = coef  * x + (inter)
            axs[i].plot(x,y, 'tomato' ,linewidth=2.5)
            
            axs[i].text(.05,.95,s = "$n$ = " + str(len(df_test)), fontsize = pfont, 
                transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            if pv <= 0.05 and pv > 0.001:
                axs[i].text(.05, .86, s = r"$p^{\bf{*}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv <= 0.001 and pv > 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{**}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv < 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{***}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            elif pv > pt:
                axs[i].text(.05, .86,s = '$p_{FDR}$ > ' + str(pt), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')


            axs[i].set_ylim([mi,mm])
            axs[i].set_xlabel(xx, fontsize = xlabel_font_size)
            axs[i].set_ylabel('$DeepSPARE$-'+r'$\bf{'+ tp+'}$', fontsize = ylabel_font_size)
            axs[i].grid(linestyle = '--')
            

        
        fig.tight_layout()
        
mm=15
mi=-5
pfont = 14
ylabel_font_size = 16
xlabel_font_size = 16

def box_plot(df,dfp, group, scores): 
    for score in scores:
        dff = df.copy()
        df_test = get_test(dff,score)
        print(score, len(df_test))

        df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD']) ) 
        df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD']))
        df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD']))

        interAD,coefAD, pvAD,r2AD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interAD','coefAD', 'pvAD_fdr','r2AD']].values[0]
        interVD,coefVD, pvVD,r2VD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interVD','coefVD', 'pvVD_fdr','r2VD']].values[0]  
        interLBD,coefLBD,pvLBD,r2LBD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interLBD','coefLBD', 'pvLBD_fdr','r2LBD']].values[0] 

        print(score)
        print("AD:",pvAD, round(coefAD,2))
        print("VD:",pvVD, round(coefVD,2))
        print("LBD:",pvLBD, round(coefLBD,2))
        fig, axs = plt.subplots(ncols = 3, nrows = 1, figsize  = (11,3))
        axs = axs.flatten()
        
    
        for i, tp, yy, pv, coef, r2,inter in zip([0,1,2],['AD','VD','LBD'],['logit_probAD', 'logit_probVD','logit_probLBD'],
                                                 [pvAD,pvVD,pvLBD],
                                                 [coefAD,coefVD,coefLBD],[r2AD,r2VD,r2LBD],[interAD,interVD,interLBD]):
            dy = df_test[yy]
            dx = df_test[score]
            ll = list(set(dx))
            for n, j in zip(ll, ll):
                axs[i].boxplot(df_test[df_test[score] == j][yy], positions = [n], notch=True, widths = 0.25 )
            axs[i].scatter(dx, dy, s = 10, color = 'skyblue', alpha= 0.8)
            max_x = dx.max()
            min_x = dx.min()
            x = np.linspace(min_x, max_x,50)
            y = coef  * x + (inter)
            axs[i].plot(x,y, 'tomato' ,linewidth=2.5)
            if pv <= 0.05 and pv > 0.001:
                axs[i].text(.05, .86, s = r"$p^{\bf{*}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv <= 0.001 and pv > 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{**}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv < 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{***}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$\beta$ = " + str(np.round(coef,3)) + \
                            "\n" + r"$R^2$ = " + str(np.round(r2,3)), fontsize = pfont, 
#                             weight='bold', 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            elif pv > pt:
                axs[i].text(.05, .86,s = '$p_{FDR}$ > ' + str(pt), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].text(.05,.95,s = "$n$ = " + str(len(df_test)), fontsize = pfont, 
                transform=axs[i].transAxes,horizontalalignment='right',verticalalignment='top',color ='black')

            axs[i].set_ylim([mi,mm])
            axs[i].set_xlabel(xx, fontsize = xlabel_font_size)
            axs[i].set_ylabel('$DeepSPARE$-'+r'$\bf{'+ tp+'}$', fontsize = ylabel_font_size)
            axs[i].grid(linestyle = '--')
        
        fig.tight_layout()

        
        

def box_plot_cohen(df,dfp, group, scores): 
    for score in scores:
        dff = df.copy()
        df_test = get_test(dff,score)
        print(score, len(df_test))

        df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD']) ) 
        df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD']))
        df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD']))


        interAD,coefAD, pvAD,r2AD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interAD','coefAD', 'pvAD_fdr','r2AD']].values[0]
        interVD,coefVD, pvVD,r2VD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interVD','coefVD', 'pvVD_fdr','r2VD']].values[0]  
        interLBD,coefLBD,pvLBD,r2LBD = dfp[(dfp['group'] == group) & (dfp['marker'] == score)][['interLBD','coefLBD', 'pvLBD_fdr','r2LBD']].values[0] 

        cohenAD = cohen_d(df_test[df_test[score] == 1]['logit_probAD'].tolist(),df_test[df_test[score] == 0]['logit_probAD'].tolist())
        cohenVD = cohen_d(df_test[df_test[score] == 1]['logit_probVD'].tolist(),df_test[df_test[score] == 0]['logit_probVD'].tolist())
        cohenLBD = cohen_d(df_test[df_test[score] == 1]['logit_probLBD'].tolist(),df_test[df_test[score] == 0]['logit_probLBD'].tolist())
        
        print(score)
        print("AD:",pvAD, round(cohenAD,2))
        print("VD:",pvVD, round(cohenVD,2))
        print("LBD:",pvLBD, round(cohenLBD,2))
        fig, axs = plt.subplots(ncols = 3, nrows = 1, figsize  = (11,3))
        axs = axs.flatten()
        

        if score == "syn":
            xx = "CSF \u03B1-synuclein"
        else:
            xx = score
        
    
        for i, tp, yy, pv, cohen  in zip([0,1,2],
                                         ['AD','VD','LBD'],
                                         ['logit_probAD', 'logit_probVD','logit_probLBD'],
                                         [pvAD,pvVD,pvLBD],
                                         [cohenAD,cohenVD,cohenLBD]):
            dy = df_test[yy]
            dx = df_test[score]
            ll = list(set(dx))
            for n, j in zip(ll, ll):
                axs[i].boxplot(df_test[df_test[score] == j][yy], positions = [n], notch=True, widths = 0.25 )
            axs[i].scatter(dx, dy, s = 10, color = 'skyblue', alpha= 0.8)

            if pv <= 0.05 and pv > 0.001:
                axs[i].text(.05, .86, s = r"$p^{\bf{*}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$Cohen'd$ = " + str(np.round(cohen,3)), fontsize = pfont,  
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv <= 0.001 and pv > 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{**}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$Cohen'd$ = " + str(np.round(cohen,3)), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')
            elif pv < 0.0001:
                axs[i].text(.05, .86, s = r"$p^{\bf{***}}_{FDR}$ = "+  r"${0:s}$".format(as_si(pv,2))+\
                            "\n" +r"$Cohen'd$ = " + str(np.round(cohen,3)), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            elif pv > pt:
                axs[i].text(.05, .86,s = '$p_{FDR}$ > ' + str(pt), fontsize = pfont, 
                        transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].text(.05,.95,s = "$n$ = " + str(len(df_test)), fontsize = pfont, 
                transform=axs[i].transAxes,horizontalalignment='left',verticalalignment='top',color ='black')

            axs[i].set_ylim([mi,mm])
            axs[i].set_xlabel(xx, fontsize = xlabel_font_size)
            axs[i].set_ylabel('$DeepSPARE$-'+r'$\bf{'+ tp+'}$', fontsize = ylabel_font_size)
            axs[i].grid(linestyle = '--')
        for i in range(0,3):
            if score == "syn":
                bars = ('negtive', 'positive')
                y_pos = np.arange(len(bars))
                axs[i].set_xticks(y_pos, bars, fontsize='13')
        
        fig.tight_layout()
        
        
        
def as_si(x, ndp):
    s = '{x:0.{ndp:d}e}'.format(x=x, ndp=ndp)
    m, e = s.split('e')
    return  r'{m:s}\times 10^{{{e:d}}}'.format(m=m, e=int(e))

In [ ]:
p = "/media/dw/Data/BINC_T1/BINC/dropbox/DW/results_GenieCode/task2_T1_ADVDLBD_Mlabel_10fold_0307v2_3DSminmax_OTF/resnet18_64_linear_mlr0.0001_ep300_batch8_ASL_gn4_gp1_c0.1_sigma0.1_wd0.001/"

df = pd.read_csv(p+"/prob_ADNI.csv")
bb = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", 'ABETA','AMYLOID_STATUS']]
bb = bb.replace('<200', 200)
bb = bb.replace('>1700', 1701)
bb['ABETA'] = bb['ABETA'].astype(float)
df = pd.merge(df,bb, on = 'mri_ID', how = 'left')
df = df.fillna(-4)
df = df.rename(columns = {'ABETA' : 'CSF_ABETA'})
df = df.rename(columns = {'AMYLOID_STATUS' : 'PET_ABETA'})


df = df.drop(columns = ['MMSE'])
df_AD = df[df.DX == 'Dementia']
df_mci = df[df.DX == 'MCI']
df_CN = df[(df.DX == 'CN')]

df_AD = df_AD[(df_AD['PET_ABETA']==1) |(df_AD['CSF_ABETA'] <= 978)]
df_mci_p = df_mci[(df_mci['PET_ABETA']==1) | (df_mci['CSF_ABETA'] <= 978)]
df_ADp = pd.concat([df_AD, df_mci_p], axis = 0)

df_mci_n = df_mci[(df_mci['PET_ABETA']==0) & (df_mci['CSF_ABETA'] > 978)]

df_CN = df_CN[(df_CN['PET_ABETA']==0)& (df_CN['CSF_ABETA'] > 978)]

print(len(df_AD),len(df_mci_p),len(df_mci_n),len(df_CN))

In [ ]:
def get_pv(df, score):
    df_test = get_test(df,score)
    df_test['logit_probAD'] = np.log(df_test['probAD']/(1-df_test['probAD']) ) 
    df_test['logit_probVD'] = np.log(df_test['probVD']/(1-df_test['probVD']))
    df_test['logit_probLBD'] = np.log(df_test['probLBD']/(1-df_test['probLBD']))
    pAD,pVD,pLBD, ymin, ymax = 'logit_probAD', 'logit_probVD','logit_probLBD', -5,5
    interAD,coefAD, pvAD = ln("AD" , df_test[score], df_test[[pAD]])
    interVD,coefVD, pvVD = ln("VD" , df_test[score], df_test[[pVD]])  
    interLBD,coefLBD,pvLBD = ln("LBD" , df_test[score], df_test[[pLBD]]) 
    r2AD = get_r2(df_test[[score]], df_test[[pAD]])
    r2VD = get_r2(df_test[[score]], df_test[[pVD]])
    r2LBD = get_r2(df_test[[score]], df_test[[pLBD]])
    if score == 'syn':
        pvAD = ttest_ind(df_test[df_test[score]==1][pAD].tolist(), df_test[df_test[score]==0][pAD].tolist())[1]
        pvVD = ttest_ind(df_test[df_test[score]==1][pVD].tolist(), df_test[df_test[score]==0][pVD].tolist())[1]
        pvLBD = ttest_ind(df_test[df_test[score]==1][pLBD].tolist(), df_test[df_test[score]==0][pLBD].tolist())[1]
    return interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD

In [ ]:


l = []
markers_AD = ['ADNI_MEM','ADNI_EF','PTAU','SPARE_TAU','syn','Hippocampus','WMH','PLASMA_NFL','PLASMAPTAU181','CSF_ptau_ab42','plasma_42']

for score in markers_AD:
    dff = df_ADp.copy()
    interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD = get_pv(dff, score)
    l.append([interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD])
    
markers_neg = ['ADNI_MEM','ADNI_EF','PTAU','syn','Hippocampus','WMH','PLASMA_NFL','PLASMAPTAU181','CSF_ptau_ab42']
for score in markers_neg:
    dff = df_mci_n.copy()
    interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD = get_pv(dff, score)
    l.append([interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD])
    
markers_CN = ['ADNI_MEM','ADNI_EF','PTAU','syn','Hippocampus','WMH','PLASMA_NFL','PLASMAPTAU181','CSF_ptau_ab42']
for score in markers_CN:
    dff = df_CN.copy()
    interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD = get_pv(dff, score)
    l.append([interAD,coefAD, pvAD,interVD,coefVD, pvVD,interLBD,coefLBD,pvLBD,r2AD,r2VD,r2LBD])

    
markers =  markers_AD +  markers_neg +  markers_CN
dfp = pd.DataFrame(l,columns = ['interAD','coefAD', 'pvAD','interVD','coefVD', 'pvVD','interLBD','coefLBD','pvLBD','r2AD','r2VD','r2LBD'])
dfp.insert(loc = 0, column = 'marker', value = markers)
pa = dfp.pvAD.tolist() +dfp.pvVD.tolist()+dfp.pvLBD.tolist()

fdr = statsmodels.stats.multitest.fdrcorrection(pa, alpha=pt, is_sorted=False)[1]

dfp.insert(loc = 1, column = 'pvAD_fdr', value =fdr[0:len(markers)])
dfp.insert(loc = 1, column = 'pvVD_fdr', value =fdr[len(markers):len(markers)*2])
dfp.insert(loc = 1, column = 'pvLBD_fdr', value =fdr[len(markers)*2:len(markers)*3])
dfp.insert(loc = 0, column = 'group', value = ['AD']*len(markers_AD) + ['MCI_neg']*len(markers_neg) + ['CN']*len(markers_CN))
 
dfp[['group','marker','pvAD','pvAD_fdr','pvVD','pvVD_fdr','pvLBD','pvLBD_fdr']]


In [ ]:
ln_plot(df_ADp, dfp,'AD',['plasma_42','ADNI_MEM','ADNI_EF','Hippocampus','WMH']) #,'ADNI_LAN','ADNI_VS'
ln_plot(df_ADp, dfp,'AD',['PLASMA_NFL','PLASMAPTAU181'])
box_plot_cohen(df_ADp, dfp,'AD',['syn'])
ln_plot(df_ADp, dfp,'AD',['CSF_ptau_ab42','PTAU','SPARE_TAU'])

In [ ]:
ln_plot(df_mci_n,dfp, 'MCI_neg', ['ADNI_MEM','ADNI_EF','Hippocampus','WMH']) #,'ADNI_LAN','ADNI_VS'
ln_plot(df_mci_n,dfp, 'MCI_neg', ['PLASMA_NFL','PLASMAPTAU181'])
box_plot_cohen(df_mci_n, dfp,'MCI_neg',['syn'])
ln_plot(df_mci_n,dfp, 'MCI_neg', ['CSF_ptau_ab42','PTAU'])

In [ ]:
ln_plot(df_CN, dfp, 'CN', ['ADNI_MEM','ADNI_EF','Hippocampus','WMH']) #,'ADNI_LAN','ADNI_VS'
ln_plot(df_CN,dfp, 'CN', ['PLASMA_NFL','PLASMAPTAU181'])
box_plot_cohen(df_CN, dfp, 'CN', ['syn'])
ln_plot(df_CN,dfp, 'CN', ['CSF_ptau_ab42','PTAU'])